In [21]:
import os
import logging
import warnings

In [22]:
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
logging.getLogger("transformers").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)


In [23]:
# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import ipywidgets as widgets
from IPython.display import display

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [24]:
# Supported languages
source_langs = {
    "English": "eng_Latn",
    "Hindi": "hin_Deva",
    "Telugu": "tel_Telu",
    "Tamil": "tam_Taml",
    "Marathi": "mar_Deva",
    "Bengali": "ben_Beng",
    "Gujarati": "guj_Gujr",
    "Punjabi": "pan_Guru",
    "German": "deu_Latn",
    "Spanish": "spa_Latn",
    "Chinese": "zho_Hans",
    "Japanese": "jpn_Jpan",
    "Korean": "kor_Hang"
}

In [25]:
# 🇮🇳 Target Bihar languages
bihari_langs = {
    "Bhojpuri": "bho_Deva",
    "Maithili": "mai_Deva",
    "Magahi": "mag_Deva",
    "Hindi": "hin_Deva",
    "Urdu": "urd_Arab"
}

In [26]:

source_lang_list = list(source_langs.keys())
bihari_lang_list = list(bihari_langs.keys())

source_lang_options = "\n".join([f"{i+1}. {lang}" for i, lang in enumerate(source_lang_list)])
bihari_lang_options = "\n".join([f"{i+1}. {lang}" for i, lang in enumerate(bihari_lang_list)])

# Widgets
source_lang_label = widgets.Label(value=f"Source Languages:\n{source_lang_options}")
src_lang_widget = widgets.Text(
    description="Enter Source Number:",
    placeholder="e.g., 1 for English"
)
target_lang_label = widgets.Label(value=f"Target (Bihar) Languages:\n{bihari_lang_options}")
tgt_lang_widget = widgets.Text(
    description="Enter Target Number:",
    placeholder="e.g., 1 for Bhojpuri"
)
text_input = widgets.Textarea(
    description="Text:",
    layout=widgets.Layout(width="100%", height="100px")
)
translate_button = widgets.Button(
    description="Translate",
    button_style="success"
)
output_box = widgets.Output()

In [27]:
def translate_text(b):
    output_box.clear_output()
    src_input = src_lang_widget.value.strip()
    tgt_input = tgt_lang_widget.value.strip()
    input_text = text_input.value.strip()

    # Validate inputs
    if not src_input.isdigit() or not 1 <= int(src_input) <= len(source_lang_list):
        with output_box:
            print(f"⚠️ Invalid source number. Enter a number between 1 and {len(source_lang_list)}.")
        return
    if not tgt_input.isdigit() or not 1 <= int(tgt_input) <= len(bihari_lang_list):
        with output_box:
            print(f"⚠️ Invalid target number. Enter a number between 1 and {len(bihari_lang_list)}.")
        return
    if not input_text:
        with output_box:
            print("⚠️ Please enter some text.")
        return

    try:
        src_lang = source_lang_list[int(src_input) - 1]
        tgt_lang = bihari_lang_list[int(tgt_input) - 1]
        src_code = source_langs[src_lang]
        tgt_code = bihari_langs[tgt_lang]
        translator = pipeline(
            "translation",
            model=model,
            tokenizer=tokenizer,
            src_lang=src_code,
            tgt_lang=tgt_code,
            max_length=512
        )
        result = translator(input_text)[0]["translation_text"]
        with output_box:
            print(f"✅ Translated to {tgt_lang}:\n{result}")
    except Exception as e:
        with output_box:
            print("❌ Translation failed:", str(e))


In [28]:
translate_button.on_click(translate_text)

In [29]:
display(source_lang_label, src_lang_widget, target_lang_label, tgt_lang_widget,
        text_input, translate_button, output_box)

Label(value='Source Languages:\n1. English\n2. Hindi\n3. Telugu\n4. Tamil\n5. Marathi\n6. Bengali\n7. Gujarati…

Text(value='', description='Enter Source Number:', placeholder='e.g., 1 for English')

Label(value='Target (Bihar) Languages:\n1. Bhojpuri\n2. Maithili\n3. Magahi\n4. Hindi\n5. Urdu')

Text(value='', description='Enter Target Number:', placeholder='e.g., 1 for Bhojpuri')

Textarea(value='', description='Text:', layout=Layout(height='100px', width='100%'))

Button(button_style='success', description='Translate', style=ButtonStyle())

Output()